# Negotiation Data Analysis Tool (v2.0)

## Features:
- 🔍 **Flexible Filtering**: Filter by demographics, scenario, model mode, etc.
- 📊 **Quick Overview**: See all successful cases with key metrics
- 🎯 **Deep Dive**: Select specific case index for detailed analysis
- 🧠 **Memory Tracking**: Track Memory (M) evolution round by round
- 📋 **Plan Tracking**: Track Plan (P) evolution round by round
- 🔄 **Integrated View**: See Transcript + Memory + Plan side-by-side
- 📈 **Comparative Analysis**: Compare multiple cases and modes

## New Database Schema:
- Removed: `student_id`, `student_name`
- Added: 10 demographic fields (optional)
- Updated: `student_goes_first` (replaced `randomize_first_turn`)

## 1. Setup and Imports

In [1]:
import sqlite3
import pandas as pd
import json
from datetime import datetime
from IPython.display import display, HTML, Markdown
import warnings
warnings.filterwarnings('ignore')

print("✅ Import successful")

✅ Import successful


## 2. Connect to Database

In [7]:
# Modify the path here
DB_PATH = "negotiations (1).db"  # Change to your database path

conn = sqlite3.connect(DB_PATH)
print(f"✅ Connected to database: {DB_PATH}")

✅ Connected to database: negotiations (1).db


## 3. Quick Database Schema Check

In [8]:
# Check table structure
schema_query = "PRAGMA table_info(negotiation_sessions)"
schema_df = pd.read_sql_query(schema_query, conn)

print("negotiation_sessions table columns:")
display(schema_df[['name', 'type']])

# Count total sessions
count_query = "SELECT COUNT(*) as total FROM negotiation_sessions"
total_sessions = pd.read_sql_query(count_query, conn)['total'][0]
print(f"\n📊 Total sessions: {total_sessions}")

negotiation_sessions table columns:


,name,type
0,session_id,TEXT
1,current_program,TEXT
2,undergrad_major,TEXT
3,gender,TEXT
4,age_range,TEXT
5,race_ethnicity,TEXT
6,work_experience,TEXT
7,first_gen_student,TEXT
8,english_proficiency,TEXT
9,negotiation_courses,TEXT



📊 Total sessions: 51


## 4. Overview Statistics

In [9]:
# Overall statistics
stats_query = """
SELECT 
    COUNT(*) as total_sessions,
    SUM(CASE WHEN deal_reached = 1 THEN 1 ELSE 0 END) as successful_deals,
    SUM(CASE WHEN deal_failed = 1 THEN 1 ELSE 0 END) as failed_deals,
    SUM(CASE WHEN status = 'active' THEN 1 ELSE 0 END) as active_sessions,
    COUNT(DISTINCT scenario_name) as unique_scenarios,
    COUNT(DISTINCT ai_model) as unique_models
FROM negotiation_sessions
"""

stats = pd.read_sql_query(stats_query, conn)

print("=" * 80)
print("                           DATABASE OVERVIEW")
print("=" * 80)
print(f"Total Sessions:          {stats['total_sessions'][0]}")
print(f"Successful Deals:        {stats['successful_deals'][0]} ({stats['successful_deals'][0]/stats['total_sessions'][0]*100:.1f}%)")
print(f"Failed Deals:            {stats['failed_deals'][0]}")
print(f"Active Sessions:         {stats['active_sessions'][0]}")
print(f"Unique Scenarios:        {stats['unique_scenarios'][0]}")
print(f"Unique Models:           {stats['unique_models'][0]}")
print("=" * 80)

                           DATABASE OVERVIEW
Total Sessions:          51
Successful Deals:        1 (2.0%)
Failed Deals:            0
Active Sessions:         50
Unique Scenarios:        2
Unique Models:           1


## 5. 🔍 Flexible Filtering - Build Your Query

Use this section to filter cases based on various criteria. Modify the filters as needed.

In [10]:
# ==================== MODIFY FILTERS HERE ====================

# Basic Filters
filter_deal_reached = True  # True, False, or None (for all)
filter_scenario = None  # e.g., "SnyderMed", "Top_talent", or None (for all)
filter_student_role = None  # "side1", "side2", or None (for all)

# Model Configuration Filters
filter_use_memory = None  # True, False, or None (for all)
filter_use_plan = None  # True, False, or None (for all)
filter_student_goes_first = None  # True, False, or None (for all)

# Demographic Filters (all optional)
filter_gender = None  # e.g., "Male", "Female", "Non-binary", or None
filter_age_range = None  # e.g., "18-25", "25-29", or None
filter_race_ethnicity = None  # e.g., "Asian", "White", or None
filter_work_experience = None  # e.g., "0-1 years", "2-3 years", or None
filter_first_gen = None  # "Yes", "No", or None
filter_english_proficiency = None  # e.g., "5 - Native-like/Fluent", or None
filter_negotiation_courses = None  # e.g., "Yes, one course", or None
filter_negotiation_experience = None  # e.g., "Frequent", or None

# ==================== BUILD QUERY ====================

base_query = """
SELECT 
    session_id,
    scenario_name,
    student_role,
    ai_model,
    student_goes_first,
    use_memory,
    use_plan,
    deal_reached,
    deal_failed,
    status,
    current_program,
    gender,
    age_range,
    race_ethnicity,
    work_experience,
    first_gen_student,
    english_proficiency,
    negotiation_courses,
    negotiation_experience,
    transcript,
    ai_memory,
    ai_plan,
    ai_memory_history,
    ai_plan_history,
    created_at,
    updated_at
FROM negotiation_sessions
WHERE 1=1
"""

conditions = []
if filter_deal_reached is not None:
    conditions.append(f"deal_reached = {1 if filter_deal_reached else 0}")
if filter_scenario:
    conditions.append(f"scenario_name = '{filter_scenario}'")
if filter_student_role:
    conditions.append(f"student_role = '{filter_student_role}'")
if filter_use_memory is not None:
    conditions.append(f"use_memory = {1 if filter_use_memory else 0}")
if filter_use_plan is not None:
    conditions.append(f"use_plan = {1 if filter_use_plan else 0}")
if filter_student_goes_first is not None:
    conditions.append(f"student_goes_first = {1 if filter_student_goes_first else 0}")
if filter_gender:
    conditions.append(f"gender = '{filter_gender}'")
if filter_age_range:
    conditions.append(f"age_range = '{filter_age_range}'")
if filter_race_ethnicity:
    conditions.append(f"race_ethnicity = '{filter_race_ethnicity}'")
if filter_work_experience:
    conditions.append(f"work_experience = '{filter_work_experience}'")
if filter_first_gen:
    conditions.append(f"first_gen_student = '{filter_first_gen}'")
if filter_english_proficiency:
    conditions.append(f"english_proficiency = '{filter_english_proficiency}'")
if filter_negotiation_courses:
    conditions.append(f"negotiation_courses = '{filter_negotiation_courses}'")
if filter_negotiation_experience:
    conditions.append(f"negotiation_experience = '{filter_negotiation_experience}'")

if conditions:
    query = base_query + " AND " + " AND ".join(conditions)
else:
    query = base_query

query += " ORDER BY created_at DESC"

# Execute query
filtered_df = pd.read_sql_query(query, conn)

print("=" * 80)
print("                           FILTER RESULTS")
print("=" * 80)
print(f"Found {len(filtered_df)} sessions matching your criteria")

if len(filtered_df) > 0:
    print("\nActive Filters:")
    if filter_deal_reached is not None:
        print(f"  - Deal Reached: {filter_deal_reached}")
    if filter_scenario:
        print(f"  - Scenario: {filter_scenario}")
    if filter_student_role:
        print(f"  - Student Role: {filter_student_role}")
    if filter_use_memory is not None:
        print(f"  - Memory Enabled: {filter_use_memory}")
    if filter_use_plan is not None:
        print(f"  - Plan Enabled: {filter_use_plan}")
    if filter_gender:
        print(f"  - Gender: {filter_gender}")
    if filter_age_range:
        print(f"  - Age Range: {filter_age_range}")
print("=" * 80)

                           FILTER RESULTS
Found 1 sessions matching your criteria

Active Filters:
  - Deal Reached: True


## 6. 📋 Quick View - List All Filtered Cases

This shows a summary table with **index numbers** for easy reference.

In [11]:
if len(filtered_df) == 0:
    print("\n❌ No cases found matching your filters")
else:
    # Create summary table
    summary_data = []
    
    for idx, row in filtered_df.iterrows():
        try:
            transcript = json.loads(row['transcript'])
            msg_count = len(transcript)
            
            start = pd.to_datetime(row['created_at'])
            end = pd.to_datetime(row['updated_at'])
            duration = (end - start).total_seconds() / 60
            
            # Model mode indicator
            if row['use_memory'] and row['use_plan']:
                mode = "M+P"
            elif row['use_memory']:
                mode = "M"
            else:
                mode = "Base"
            
            summary_data.append({
                'Index': len(summary_data),
                'Session_ID': row['session_id'][:8],
                'Scenario': row['scenario_name'],
                'Role': row['student_role'],
                'Mode': mode,
                'Goes_First': '✅' if row['student_goes_first'] else '❌',
                'Deal': '✅' if row['deal_reached'] else '❌',
                'Msgs': msg_count,
                'Rounds': msg_count // 2,
                'Duration_min': f"{duration:.1f}",
                'Gender': row['gender'] if row['gender'] else '-',
                'Age': row['age_range'] if row['age_range'] else '-',
                'Experience': row['negotiation_experience'][:20] if row['negotiation_experience'] else '-'
            })
        except Exception as e:
            print(f"Error processing row: {e}")
            continue
    
    if summary_data:
        summary_df = pd.DataFrame(summary_data)
        
        print("\n" + "=" * 120)
        print("                                    FILTERED CASES SUMMARY")
        print("=" * 120)
        print(f"Total: {len(summary_df)} cases")
        print(f"Successful: {summary_df['Deal'].str.contains('✅').sum()} cases")
        print("\nUse the 'Index' number below to analyze specific cases in detail (see Section 7-11)")
        print("=" * 120)
        
        display(summary_df)
        
        # Store for later use
        global_filtered_df = filtered_df.copy()
        global_filtered_df['display_index'] = range(len(global_filtered_df))
    else:
        print("\n❌ Error processing data")


                                    FILTERED CASES SUMMARY
Total: 1 cases
Successful: 1 cases

Use the 'Index' number below to analyze specific cases in detail (see Section 7-11)


,Index,Session_ID,Scenario,Role,Mode,Goes_First,Deal,Msgs,Rounds,Duration_min,Gender,Age,Experience
0,0,778ca358,Main_Street,side2,M+P,❌,✅,12,6,55.5,-,-,-


## 7. 🎯 Deep Dive - Analyze Specific Case by Index

Enter the **Index** number from the table above to see detailed analysis.

In [12]:
# ==================== ENTER INDEX HERE ====================
CASE_INDEX = 0  # Change this to the index you want to analyze
# ==========================================================

if len(filtered_df) == 0:
    print("\n❌ No filtered data available. Run Section 5 first.")
elif CASE_INDEX >= len(filtered_df):
    print(f"\n❌ Invalid index. Please choose 0-{len(filtered_df)-1}")
else:
    case = global_filtered_df[global_filtered_df['display_index'] == CASE_INDEX].iloc[0]
    
    print("\n" + "=" * 80)
    print(f"                    CASE ANALYSIS - Index {CASE_INDEX}")
    print("=" * 80)
    
    # Basic Info
    print("\n📋 BASIC INFORMATION:")
    print("-" * 80)
    print(f"Session ID:          {case['session_id']}")
    print(f"Scenario:            {case['scenario_name']}")
    print(f"Student Role:        {case['student_role']}")
    print(f"AI Model:            {case['ai_model']}")
    print(f"Student Goes First:  {'Yes' if case['student_goes_first'] else 'No'}")
    print(f"Memory Enabled:      {'Yes' if case['use_memory'] else 'No'}")
    print(f"Plan Enabled:        {'Yes' if case['use_plan'] else 'No'}")
    print(f"Deal Reached:        {'✅ Yes' if case['deal_reached'] else '❌ No'}")
    print(f"Status:              {case['status']}")
    
    # Demographics
    print("\n👤 DEMOGRAPHICS:")
    print("-" * 80)
    print(f"Program:             {case['current_program'] if case['current_program'] else 'Not disclosed'}")
    print(f"Gender:              {case['gender'] if case['gender'] else 'Not disclosed'}")
    print(f"Age Range:           {case['age_range'] if case['age_range'] else 'Not disclosed'}")
    print(f"Race/Ethnicity:      {case['race_ethnicity'] if case['race_ethnicity'] else 'Not disclosed'}")
    print(f"Work Experience:     {case['work_experience'] if case['work_experience'] else 'Not disclosed'}")
    print(f"First Gen:           {case['first_gen_student'] if case['first_gen_student'] else 'Not disclosed'}")
    print(f"English Level:       {case['english_proficiency'] if case['english_proficiency'] else 'Not disclosed'}")
    print(f"Negotiation Courses: {case['negotiation_courses'] if case['negotiation_courses'] else 'Not disclosed'}")
    print(f"Negotiation Exp:     {case['negotiation_experience'] if case['negotiation_experience'] else 'Not disclosed'}")
    
    # Timing
    print("\n⏱️  TIMING:")
    print("-" * 80)
    print(f"Created:             {case['created_at']}")
    print(f"Updated:             {case['updated_at']}")
    
    try:
        start = pd.to_datetime(case['created_at'])
        end = pd.to_datetime(case['updated_at'])
        duration = (end - start).total_seconds() / 60
        print(f"Duration:            {duration:.1f} minutes")
    except:
        pass
    
    # Transcript Analysis
    try:
        transcript = json.loads(case['transcript'])
        print("\n💬 TRANSCRIPT ANALYSIS:")
        print("-" * 80)
        print(f"Total Messages:      {len(transcript)}")
        print(f"Completed Rounds:    {len(transcript) // 2}")
        
        # Count deal signals
        deal_signals = sum(1 for msg in transcript if '$DEAL_REACHED$' in msg)
        print(f"$DEAL_REACHED$:      {deal_signals} occurrence(s)")
        
        # Message lengths
        message_lengths = []
        for msg in transcript:
            if ' - ' in msg:
                content = msg.split(' - ', 1)[1]
                if ': ' in content:
                    content = content.split(': ', 1)[1]
                message_lengths.append(len(content))
        
        if message_lengths:
            print(f"Avg Message Length:  {sum(message_lengths)/len(message_lengths):.0f} characters")
            print(f"Shortest Message:    {min(message_lengths)} characters")
            print(f"Longest Message:     {max(message_lengths)} characters")
        
        # Per-round timing
        try:
            rounds = len(transcript) // 2
            if rounds > 0 and duration > 0:
                print(f"Avg Time/Round:      {duration/rounds:.1f} minutes")
                print(f"Avg Time/Message:    {duration/len(transcript):.1f} minutes")
        except:
            pass
            
    except Exception as e:
        print(f"\n❌ Transcript analysis error: {e}")
    
    print("\n" + "=" * 80)


                    CASE ANALYSIS - Index 0

📋 BASIC INFORMATION:
--------------------------------------------------------------------------------
Session ID:          778ca358-dad7-4982-8f68-e41bf889ca46
Scenario:            Main_Street
Student Role:        side2
AI Model:            openai/gpt-5
Student Goes First:  No
Memory Enabled:      Yes
Plan Enabled:        Yes
Deal Reached:        ✅ Yes
Status:              completed

👤 DEMOGRAPHICS:
--------------------------------------------------------------------------------
Program:             1
Gender:              Not disclosed
Age Range:           Not disclosed
Race/Ethnicity:      Not disclosed
Work Experience:     Not disclosed
First Gen:           Not disclosed
English Level:       Not disclosed
Negotiation Courses: Not disclosed
Negotiation Exp:     Not disclosed

⏱️  TIMING:
--------------------------------------------------------------------------------
Created:             2025-12-09T05:19:13.133135
Updated:             2025

## 8. 🧠 Memory (M) Evolution - Track Round by Round

In [13]:
# Use same CASE_INDEX as Section 7

if len(filtered_df) == 0:
    print("\n❌ No filtered data available. Run Section 5 first.")
elif CASE_INDEX >= len(filtered_df):
    print(f"\n❌ Invalid index. Please choose 0-{len(filtered_df)-1}")
else:
    case = global_filtered_df[global_filtered_df['display_index'] == CASE_INDEX].iloc[0]
    
    print("\n" + "=" * 80)
    print(f"              MEMORY (M) EVOLUTION - Index {CASE_INDEX}")
    print("=" * 80)
    print(f"Session: {case['session_id']}")
    print(f"Memory Enabled: {'Yes' if case['use_memory'] else 'No'}")
    print("=" * 80)
    
    if not case['use_memory']:
        print("\n❌ Memory not enabled for this case")
    else:
        try:
            # Parse memory history
            memory_history = json.loads(case['ai_memory_history']) if case['ai_memory_history'] else []
            
            if not memory_history:
                print("\n⚠️  No memory history recorded")
            else:
                print(f"\n📊 Total Memory Updates: {len(memory_history)}")
                print("=" * 80)
                
                for i, memory_entry in enumerate(memory_history, 1):
                    round_num = memory_entry.get('round', f'Update {i}')
                    content = memory_entry.get('content', 'No content')
                    
                    print(f"\n🧠 MEMORY STATE - Round {round_num}")
                    print("-" * 80)
                    print(content)
                    print("-" * 80)
                
                # Final memory state
                if case['ai_memory']:
                    print(f"\n\n💾 FINAL MEMORY STATE:")
                    print("=" * 80)
                    print(case['ai_memory'])
                    
                    # Analysis
                    print("\n\n📈 MEMORY ANALYSIS:")
                    print("-" * 80)
                    print(f"Total updates: {len(memory_history)}")
                    print(f"Final memory length: {len(case['ai_memory'])} characters")
                    print(f"Final memory lines: {len(case['ai_memory'].split(chr(10)))}")
                    
                    # Check for key sections
                    keywords = ['OFFERS', 'PATTERNS', 'PRIORITIES', 'CONSTRAINTS']
                    found = [kw for kw in keywords if kw in case['ai_memory']]
                    if found:
                        print(f"Key sections present: {', '.join(found)}")
                    
        except Exception as e:
            print(f"\n❌ Error parsing memory history: {e}")
            if case['ai_memory_history']:
                print(f"Raw ai_memory_history (first 200 chars): {str(case['ai_memory_history'])[:200]}...")


              MEMORY (M) EVOLUTION - Index 0
Session: 778ca358-dad7-4982-8f68-e41bf889ca46
Memory Enabled: Yes

📊 Total Memory Updates: 5

🧠 MEMORY STATE - Round 2.1
--------------------------------------------------------------------------------
OFFERS: [Us: Price: 480,000 [unchanged]; Them: Price: 650,000 [unchanged]; Them-best-for-us: Price: 650,000 (only seller offer so far)]
OPPONENT PATTERNS: [Rejected 480,000; anchored at 650,000; justifies value via comps (550–600), mixed-use zoning, Main St location, income; states no rush and willingness to hold or pass to family; asked our true willingness/price range]
OPPONENT PRIORITIES: [Maximize price near perceived FMV (Stated); Emphasis on comps 550–600 and asset qualities (Stated); Low urgency/timing flexibility (Stated)]
OPPONENT CONSTRAINTS: [Will not accept 480,000 (Stated via rejection); No urgency—comfortable holding or passing to family (Stated); Minimum acceptable price not stated; Deal conditioned on “right number” aligning w

## 9. 📋 Plan (P) Evolution - Track Round by Round

In [14]:
# Use same CASE_INDEX as Section 7

if len(filtered_df) == 0:
    print("\n❌ No filtered data available. Run Section 5 first.")
elif CASE_INDEX >= len(filtered_df):
    print(f"\n❌ Invalid index. Please choose 0-{len(filtered_df)-1}")
else:
    case = global_filtered_df[global_filtered_df['display_index'] == CASE_INDEX].iloc[0]
    
    print("\n" + "=" * 80)
    print(f"              PLAN (P) EVOLUTION - Index {CASE_INDEX}")
    print("=" * 80)
    print(f"Session: {case['session_id']}")
    print(f"Plan Enabled: {'Yes' if case['use_plan'] else 'No'}")
    print("=" * 80)
    
    if not case['use_plan']:
        print("\n❌ Plan not enabled for this case")
    else:
        try:
            # Parse plan history
            plan_history = json.loads(case['ai_plan_history']) if case['ai_plan_history'] else []
            
            if not plan_history:
                print("\n⚠️  No plan history recorded")
            else:
                print(f"\n📊 Total Plan Updates: {len(plan_history)}")
                print("=" * 80)
                
                for i, plan_entry in enumerate(plan_history, 1):
                    round_num = plan_entry.get('round', f'Update {i}')
                    content = plan_entry.get('content', 'No content')
                    
                    print(f"\n📋 STRATEGIC PLAN - Round {round_num}")
                    print("-" * 80)
                    print(content)
                    print("-" * 80)
                
                # Final plan state
                if case['ai_plan']:
                    print(f"\n\n💾 FINAL PLAN STATE:")
                    print("=" * 80)
                    print(case['ai_plan'])
                    
                    # Analysis
                    print("\n\n📈 PLAN ANALYSIS:")
                    print("-" * 80)
                    print(f"Total updates: {len(plan_history)}")
                    print(f"Final plan length: {len(case['ai_plan'])} characters")
                    print(f"Final plan lines: {len(case['ai_plan'].split(chr(10)))}")
                    
                    # Check for strategy sections
                    keywords = ['ROUND GOAL', 'KEY LEVERS', 'TACTICS', 'OFFER SCAFFOLD', 'RISK']
                    found = [kw for kw in keywords if kw in case['ai_plan']]
                    if found:
                        print(f"Strategy sections present: {', '.join(found)}")
                    
        except Exception as e:
            print(f"\n❌ Error parsing plan history: {e}")
            if case['ai_plan_history']:
                print(f"Raw ai_plan_history (first 200 chars): {str(case['ai_plan_history'])[:200]}...")


              PLAN (P) EVOLUTION - Index 0
Session: 778ca358-dad7-4982-8f68-e41bf889ca46
Plan Enabled: Yes

📊 Total Plan Updates: 6

📋 STRATEGIC PLAN - Round 1.1
--------------------------------------------------------------------------------
- ROUND GOAL: Set a firm low anchor and surface Rosalind’s motivation/timeline; get her target price while positioning myself as the fastest, cleanest buyer.
- KEY LEVERS: Price; market softness; required conversion/capex; vacancy/retail risk; speed/certainty (as narrative leverage).
- TACTICS: Open warmly; anchor decisively at a below-rental-value number with brief, reasoned rationale; ask 2–3 probing questions (timeline, other offers, urgency); emphasize I can move fast at the right price; avoid revealing adjacency synergy or my max; invite her counter.
- OFFER SCAFFOLD: $480,000
- ANCHOR RATIONALE: Lobster-led local softness; triplex rents support low-500s; heritage/retail repositioning costs; commercial conversion risk.
- INFO PROBES: “What t

## 10. 🔄 Integrated View - Transcript + Memory + Plan Side-by-Side

In [15]:
# Use same CASE_INDEX as Section 7
# See how M and P evolved alongside the conversation

def display_round(round_num, messages, memory_by_round, plan_by_round):
    """Helper function to display one round's information"""
    print(f"\n{'='*100}")
    print(f"                                 ROUND {round_num}")
    print(f"{'='*100}")
    
    # Messages
    print(f"\n💬 MESSAGES:")
    print("-" * 100)
    for msg in messages:
        print(msg)
    
    # Memory (if available for this round)
    memory_key = f"{round_num}.2"  # AI's turn
    if memory_key in memory_by_round:
        print(f"\n🧠 MEMORY UPDATE (Round {memory_key}):")
        print("-" * 100)
        mem_content = memory_by_round[memory_key]
        if len(mem_content) > 500:
            print(mem_content[:500] + "...")
            print(f"\n[Memory truncated - Full length: {len(mem_content)} chars]")
        else:
            print(mem_content)
    
    # Plan (if available for this round)
    plan_key = f"{round_num}.2"  # AI's turn
    if plan_key in plan_by_round:
        print(f"\n📋 PLAN UPDATE (Round {plan_key}):")
        print("-" * 100)
        plan_content = plan_by_round[plan_key]
        if len(plan_content) > 500:
            print(plan_content[:500] + "...")
            print(f"\n[Plan truncated - Full length: {len(plan_content)} chars]")
        else:
            print(plan_content)
    
    print("\n" + "-" * 100)


if len(filtered_df) == 0:
    print("\n❌ No filtered data available. Run Section 5 first.")
elif CASE_INDEX >= len(filtered_df):
    print(f"\n❌ Invalid index. Please choose 0-{len(filtered_df)-1}")
else:
    case = global_filtered_df[global_filtered_df['display_index'] == CASE_INDEX].iloc[0]
    
    print("\n" + "=" * 100)
    print(f"              INTEGRATED VIEW: Transcript + Memory + Plan - Index {CASE_INDEX}")
    print("=" * 100)
    
    try:
        transcript = json.loads(case['transcript'])
        memory_history = json.loads(case['ai_memory_history']) if case['ai_memory_history'] else []
        plan_history = json.loads(case['ai_plan_history']) if case['ai_plan_history'] else []
        
        # Build round mapping
        memory_by_round = {m.get('round'): m.get('content') for m in memory_history}
        plan_by_round = {p.get('round'): p.get('content') for p in plan_history}
        
        # Group transcript by rounds
        current_round = 1
        round_messages = []
        
        for msg in transcript:
            # Extract round number from message (e.g., "Round 1.1 - ...")
            if msg.startswith('Round '):
                parts = msg.split(' - ', 1)
                if len(parts) > 0:
                    round_info = parts[0].replace('Round ', '')
                    try:
                        round_num = int(round_info.split('.')[0])
                        
                        if round_num != current_round:
                            # Display previous round
                            if round_messages:
                                display_round(current_round, round_messages, memory_by_round, plan_by_round)
                            current_round = round_num
                            round_messages = []
                        
                        round_messages.append(msg)
                    except:
                        round_messages.append(msg)
        
        # Display last round
        if round_messages:
            display_round(current_round, round_messages, memory_by_round, plan_by_round)
            
    except Exception as e:
        print(f"\n❌ Error creating integrated view: {e}")
        import traceback
        traceback.print_exc()


              INTEGRATED VIEW: Transcript + Memory + Plan - Index 0

                                 ROUND 1

💬 MESSAGES:
----------------------------------------------------------------------------------------------------
Round 1.1 - Fred Starr (Buyer): Rosalind, good to see you. The old Christmas Elves spot—lots of memories on that block, even if I was never much for tinsel.

Straight to it: I’m a real buyer and I can move quickly if the number makes sense. Given where Yarmouth is right now—lobster prices soft, financing tighter—and the work I’d want to do to refresh and reposition, I’m at $480,000 for 123 Main.

That’s based on it penciling as a straight rental in the low 500s at best, plus capex and lease-up risk on the commercial component. At $480k I can keep this simple.

What timing are you looking for? Are there any other serious offers on the table? And if I make this easy for you, what price range would actually get a yes from you?
Round 1.2 - Rosalind Cain (Seller): Fred,

## 11. 📜 View Full Transcript (by Index)

In [16]:
# Use same CASE_INDEX as Section 7

if len(filtered_df) == 0:
    print("\n❌ No filtered data available. Run Section 5 first.")
elif CASE_INDEX >= len(filtered_df):
    print(f"\n❌ Invalid index. Please choose 0-{len(filtered_df)-1}")
else:
    case = global_filtered_df[global_filtered_df['display_index'] == CASE_INDEX].iloc[0]
    
    print("\n" + "=" * 80)
    print(f"              FULL TRANSCRIPT - Index {CASE_INDEX}")
    print("=" * 80)
    print(f"Session: {case['session_id']}")
    print(f"Scenario: {case['scenario_name']}")
    print("=" * 80)
    
    try:
        transcript = json.loads(case['transcript'])
        
        for i, message in enumerate(transcript, 1):
            print(f"\n[Message {i}]")
            print(message)
            print("-" * 80)
            
    except Exception as e:
        print(f"\n❌ Error displaying transcript: {e}")


              FULL TRANSCRIPT - Index 0
Session: 778ca358-dad7-4982-8f68-e41bf889ca46
Scenario: Main_Street

[Message 1]
Round 1.1 - Fred Starr (Buyer): Rosalind, good to see you. The old Christmas Elves spot—lots of memories on that block, even if I was never much for tinsel.

Straight to it: I’m a real buyer and I can move quickly if the number makes sense. Given where Yarmouth is right now—lobster prices soft, financing tighter—and the work I’d want to do to refresh and reposition, I’m at $480,000 for 123 Main.

That’s based on it penciling as a straight rental in the low 500s at best, plus capex and lease-up risk on the commercial component. At $480k I can keep this simple.

What timing are you looking for? Are there any other serious offers on the table? And if I make this easy for you, what price range would actually get a yes from you?
--------------------------------------------------------------------------------

[Message 2]
Round 1.2 - Rosalind Cain (Seller): Fred, I apprec

## 12. 📊 Comparative Analysis - Multiple Cases

In [ ]:
# ==================== ENTER INDICES HERE ====================
COMPARE_INDICES = [0, 1, 2]  # Change to indices you want to compare
# ===========================================================

if len(filtered_df) == 0:
    print("\n❌ No filtered data available. Run Section 5 first.")
else:
    valid_indices = [i for i in COMPARE_INDICES if i < len(filtered_df)]
    
    if not valid_indices:
        print(f"\n❌ No valid indices. Choose from 0-{len(filtered_df)-1}")
    else:
        print("\n" + "=" * 100)
        print(f"                    COMPARATIVE ANALYSIS - {len(valid_indices)} Cases")
        print("=" * 100)
        
        comparison_data = []
        
        for idx in valid_indices:
            case = global_filtered_df[global_filtered_df['display_index'] == idx].iloc[0]
            
            try:
                transcript = json.loads(case['transcript'])
                msg_count = len(transcript)
                
                start = pd.to_datetime(case['created_at'])
                end = pd.to_datetime(case['updated_at'])
                duration = (end - start).total_seconds() / 60
                
                mode = "M+P" if case['use_memory'] and case['use_plan'] else ("M" if case['use_memory'] else "Base")
                
                comparison_data.append({
                    'Index': idx,
                    'Session': case['session_id'][:8],
                    'Scenario': case['scenario_name'],
                    'Role': case['student_role'],
                    'Mode': mode,
                    'First': 'Student' if case['student_goes_first'] else 'AI',
                    'Deal': '✅' if case['deal_reached'] else '❌',
                    'Rounds': msg_count // 2,
                    'Duration': f"{duration:.1f}min",
                    'Gender': case['gender'][:10] if case['gender'] else '-',
                    'Experience': case['negotiation_experience'][:15] if case['negotiation_experience'] else '-'
                })
            except:
                continue
        
        if comparison_data:
            comp_df = pd.DataFrame(comparison_data)
            display(comp_df)
        else:
            print("\n❌ Error processing comparison data")

## 13. 📈 Demographic Distribution Analysis

In [ ]:
# Analyze demographic distributions in filtered data

if len(filtered_df) == 0:
    print("\n❌ No filtered data available.")
else:
    print("\n" + "=" * 80)
    print("                    DEMOGRAPHIC DISTRIBUTION")
    print("=" * 80)
    
    # Gender distribution
    print("\n👤 Gender Distribution:")
    gender_counts = filtered_df['gender'].value_counts(dropna=False)
    for gender, count in gender_counts.items():
        gender_label = gender if gender else 'Not disclosed'
        print(f"  {gender_label}: {count} ({count/len(filtered_df)*100:.1f}%)")
    
    # Age range distribution
    print("\n📅 Age Range Distribution:")
    age_counts = filtered_df['age_range'].value_counts(dropna=False)
    for age, count in age_counts.items():
        age_label = age if age else 'Not disclosed'
        print(f"  {age_label}: {count} ({count/len(filtered_df)*100:.1f}%)")
    
    # Negotiation experience distribution
    print("\n💼 Negotiation Experience Distribution:")
    exp_counts = filtered_df['negotiation_experience'].value_counts(dropna=False)
    for exp, count in exp_counts.items():
        exp_label = exp if exp else 'Not disclosed'
        print(f"  {exp_label}: {count} ({count/len(filtered_df)*100:.1f}%)")
    
    # Success rate by experience
    print("\n✅ Success Rate by Negotiation Experience:")
    for exp in filtered_df['negotiation_experience'].dropna().unique():
        exp_df = filtered_df[filtered_df['negotiation_experience'] == exp]
        success_rate = exp_df['deal_reached'].sum() / len(exp_df) * 100
        print(f"  {exp}: {success_rate:.1f}% ({exp_df['deal_reached'].sum()}/{len(exp_df)})")
    
    print("\n" + "=" * 80)

## 14. 🎯 Success Rate Analysis by Various Factors

In [ ]:
# Analyze success rates across different dimensions

if len(filtered_df) == 0:
    print("\n❌ No filtered data available.")
else:
    print("\n" + "=" * 80)
    print("                    SUCCESS RATE ANALYSIS")
    print("=" * 80)
    
    # By scenario
    print("\n📋 Success Rate by Scenario:")
    for scenario in filtered_df['scenario_name'].unique():
        scenario_df = filtered_df[filtered_df['scenario_name'] == scenario]
        success_rate = scenario_df['deal_reached'].sum() / len(scenario_df) * 100
        print(f"  {scenario}: {success_rate:.1f}% ({scenario_df['deal_reached'].sum()}/{len(scenario_df)})")
    
    # By model mode
    print("\n🤖 Success Rate by Model Mode:")
    for memory in [True, False]:
        for plan in [True, False]:
            mode_df = filtered_df[(filtered_df['use_memory'] == memory) & (filtered_df['use_plan'] == plan)]
            if len(mode_df) > 0:
                mode_label = "M+P" if memory and plan else ("M" if memory else "Base")
                success_rate = mode_df['deal_reached'].sum() / len(mode_df) * 100
                print(f"  {mode_label}: {success_rate:.1f}% ({mode_df['deal_reached'].sum()}/{len(mode_df)})")
    
    # By who goes first
    print("\n👤 Success Rate by Who Goes First:")
    student_first = filtered_df[filtered_df['student_goes_first'] == True]
    ai_first = filtered_df[filtered_df['student_goes_first'] == False]
    
    if len(student_first) > 0:
        rate = student_first['deal_reached'].sum() / len(student_first) * 100
        print(f"  Student First: {rate:.1f}% ({student_first['deal_reached'].sum()}/{len(student_first)})")
    
    if len(ai_first) > 0:
        rate = ai_first['deal_reached'].sum() / len(ai_first) * 100
        print(f"  AI First: {rate:.1f}% ({ai_first['deal_reached'].sum()}/{len(ai_first)})")
    
    # By role
    print("\n🎭 Success Rate by Role:")
    for role in filtered_df['student_role'].unique():
        role_df = filtered_df[filtered_df['student_role'] == role]
        success_rate = role_df['deal_reached'].sum() / len(role_df) * 100
        print(f"  {role}: {success_rate:.1f}% ({role_df['deal_reached'].sum()}/{len(role_df)})")
    
    print("\n" + "=" * 80)

## 15. 📊 Memory & Plan Statistics Comparison

In [ ]:
# Compare M vs M+P modes across all filtered cases

if len(filtered_df) == 0:
    print("\n❌ No filtered data available.")
else:
    print("\n" + "=" * 80)
    print("              MEMORY & PLAN USAGE STATISTICS")
    print("=" * 80)
    
    # Group by mode
    base_mode = filtered_df[(filtered_df['use_memory'] == False) & (filtered_df['use_plan'] == False)]
    m_mode = filtered_df[(filtered_df['use_memory'] == True) & (filtered_df['use_plan'] == False)]
    mp_mode = filtered_df[(filtered_df['use_memory'] == True) & (filtered_df['use_plan'] == True)]
    
    modes = [
        ('Base (No M, No P)', base_mode),
        ('M Only', m_mode),
        ('M+P', mp_mode)
    ]
    
    for mode_name, mode_df in modes:
        if len(mode_df) > 0:
            print(f"\n📊 {mode_name}:")
            print("-" * 80)
            print(f"  Total cases: {len(mode_df)}")
            print(f"  Success rate: {mode_df['deal_reached'].sum() / len(mode_df) * 100:.1f}%")
            
            # Average rounds
            try:
                avg_msgs = 0
                for _, row in mode_df.iterrows():
                    transcript = json.loads(row['transcript'])
                    avg_msgs += len(transcript)
                avg_msgs /= len(mode_df)
                print(f"  Avg messages: {avg_msgs:.1f}")
                print(f"  Avg rounds: {avg_msgs/2:.1f}")
            except:
                pass
            
            # Average duration
            try:
                total_duration = 0
                for _, row in mode_df.iterrows():
                    start = pd.to_datetime(row['created_at'])
                    end = pd.to_datetime(row['updated_at'])
                    total_duration += (end - start).total_seconds() / 60
                avg_duration = total_duration / len(mode_df)
                print(f"  Avg duration: {avg_duration:.1f} minutes")
            except:
                pass
    
    print("\n" + "=" * 80)

## 16. 💾 Export Filtered Data to CSV

In [ ]:
# Export filtered results for further analysis

if len(filtered_df) == 0:
    print("\n❌ No data to export.")
else:
    # Select columns to export
    export_cols = [
        'session_id', 'scenario_name', 'student_role', 'ai_model',
        'student_goes_first', 'use_memory', 'use_plan',
        'deal_reached', 'deal_failed', 'status',
        'current_program', 'gender', 'age_range', 'race_ethnicity',
        'work_experience', 'first_gen_student', 'english_proficiency',
        'negotiation_courses', 'negotiation_experience',
        'created_at', 'updated_at'
    ]
    
    export_df = filtered_df[export_cols].copy()
    
    # Generate filename
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"negotiation_data_{timestamp}.csv"
    
    export_df.to_csv(filename, index=False)
    print(f"\n✅ Exported {len(export_df)} records to: {filename}")

## 17. 🔍 Quick Search - Find Specific Sessions

In [ ]:
# Search by session ID or partial match

SEARCH_SESSION_ID = ""  # Enter full or partial session ID

if SEARCH_SESSION_ID:
    search_query = f"""
    SELECT session_id, scenario_name, deal_reached, created_at
    FROM negotiation_sessions
    WHERE session_id LIKE '%{SEARCH_SESSION_ID}%'
    """
    
    search_results = pd.read_sql_query(search_query, conn)
    
    if len(search_results) > 0:
        print(f"\n✅ Found {len(search_results)} matching session(s):")
        display(search_results)
    else:
        print(f"\n❌ No sessions found matching '{SEARCH_SESSION_ID}'")
else:
    print("\n⚠️  Enter a session ID to search")

## 18. 🗑️ Close Database Connection

In [ ]:
conn.close()
print("✅ Database connection closed")